In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
# import sys
# !{sys.executable} -m pip install optuna

In [3]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

Using TensorFlow backend.


In [4]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
pd.set_option("display.max_rows", 500)

In [5]:
csv_file_folder =  '../../data/input'

file_folder =  '../../data/feature'

In [8]:
mytrial = []

In [18]:
columns = df_train2.columns.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'])

X_data = df_train2.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'], axis=1).values.astype('float32')
y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=128)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [19]:
columns

Index(['atom_index_0', 'atom_index_1', 'type', 'NH', 'inv_dist1E', 'linkM1',
       'ID', 'adN2', 'yukawa_H.y', 'coulomb_H.x',
       ...
       'atom_index_1_eigv_gap', 'atom_index_1_fiedler_eig',
       'atom_index_1_connectedness', 'atom_index_1_sv_min',
       'atom_index_1_coulomb_mean', 'atom_index_1_sv_0', 'atom_index_1_sv_1',
       'atom_index_1_sv_2', 'atom_index_1_sv_3', 'atom_index_1_sv_4'],
      dtype='object', length=770)

In [7]:
LGB_PARAMS = {
    'objective': 'regression',
    'metric': 'mae',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'learning_rate': 0.2,
    'num_leaves': 256,
    'min_child_samples': 79,
    'max_depth': 9,
    'subsample_freq': 1,
    'subsample': 0.9,
    'bagging_seed': 11,
    'reg_alpha': 0.1,
    'reg_lambda': 0.3,
    'colsample_bytree': 1.0
}

In [13]:
model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
model.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae',
        verbose=100, early_stopping_rounds=200)

y_pred = model.predict(X_val)
np.log(mean_absolute_error(y_val, y_pred))

Training until validation scores don't improve for 200 rounds.
[100]	training's l1: 0.843619	valid_1's l1: 0.941455
[200]	training's l1: 0.685367	valid_1's l1: 0.832333
[300]	training's l1: 0.594306	valid_1's l1: 0.782328
[400]	training's l1: 0.530607	valid_1's l1: 0.751125
[500]	training's l1: 0.478117	valid_1's l1: 0.728364
[600]	training's l1: 0.437059	valid_1's l1: 0.712559
[700]	training's l1: 0.40351	valid_1's l1: 0.701606
[800]	training's l1: 0.372437	valid_1's l1: 0.691977
[900]	training's l1: 0.345029	valid_1's l1: 0.684375
[1000]	training's l1: 0.320128	valid_1's l1: 0.6775
[1100]	training's l1: 0.297154	valid_1's l1: 0.672255
[1200]	training's l1: 0.277357	valid_1's l1: 0.667895
[1300]	training's l1: 0.258877	valid_1's l1: 0.663758
[1400]	training's l1: 0.242174	valid_1's l1: 0.660598
[1500]	training's l1: 0.227386	valid_1's l1: 0.657729
Did not meet early stopping. Best iteration is:
[1500]	training's l1: 0.227386	valid_1's l1: 0.657729


-0.4189623189116563

In [50]:
perm = PermutationImportance(model, random_state=42).fit(X_val, y_val)
df_feature_importances_i2 = eli5.explain_weights_dfs(perm)['feature_importances']
df_feature_importances_i2['feature2'] = df_feature_importances_i2['feature'].apply(lambda x : columns[int(x.replace('x',''))])
df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['weight'], ascending=False)
df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)

In [61]:
columns_comb =[]
best_error = 999
his = []
for col in df_feature_importances_i2.feature2.tolist():
    
    X_data = df_train2[[col]+columns_comb].values.astype('float32')
    y_data = df_train2['scalar_coupling_constant'].values.astype('float32')
    
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=128)
    
    model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
    model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
    y_pred = model.predict(X_val)
    y_pred_train = model.predict(X_train)
    error = np.log(mean_absolute_error(y_val, y_pred))
    error_train = np.log(mean_absolute_error(y_train, y_pred_train))
    print('col', [col]+columns_comb, 'error', error, 'error_train', error_train)
    
    if error < best_error:
        best_error = error
        columns_comb += [col]
    his.append({'col':[col]+columns_comb, 'error':error, 'error_train':error_train})
    
    if len(columns_comb)>70:
        break
    

col ['tertiary_angle_0'] error 1.6276652945323087 error_train 1.6261026767490179
col ['atom_1_bond_lengths_mean', 'tertiary_angle_0'] error 1.1251459463353315 error_train 1.1111938950459355
col ['inv_dist1R', 'tertiary_angle_0', 'atom_1_bond_lengths_mean'] error 0.9434274711113362 error_train 0.8938767670030703
col ['yukawa_H.y', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R'] error 0.8742410993931745 error_train 0.8004909545883621
col ['d_4_3', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y'] error 0.7738920824296573 error_train 0.6830555912733778
col ['mulliken_atom_0', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3'] error 0.5050918243835566 error_train 0.26942102956554553
col ['dist_O_0_x', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0'] error 0.2989170075313587 error_train 0.04023621532048349
col ['dist_to_type_0_mean', 'tertiary_angle_0', 'atom_1_bo

col ['atom_index_1_dist_q.01', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x'] error -0.23083099168447394 error_train -0.661367654371506
col ['cos_center1', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x'] error -0.26370041498548374 error_train -0.7124493752946607
col ['dist_C_2_x', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_di

col ['dist_C_1_y', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div'] error -0.4160091924309197 error_train -0.9318140304801948
col ['dist_N_0_x', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist

col ['molecule_atom_index_1_dist_min_diff', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4'] error -0.43252048120987213 error_train -0.9660190082807623
col ['dist_to_type_mean', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1

col ['dist_to_type_std', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1'] error -0.4409291868459885 error_train -0.974547325673199
col ['tertiary_angle_5', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken

...

col ['d_9_2', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0', 'tertiary_an

col ['adH1', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0', 'tertiary_ang

col ['tertiary_distance_10', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0

col ['atom_index_1_sv_min', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0'

col ['range_dist_no_bond_x', 'tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0

In [66]:
X_data = df_train2[columns_comb].values.astype('float32')
y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=128)

model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
y_pred = model.predict(X_val)
y_pred_train = model.predict(X_train)
error = np.log(mean_absolute_error(y_val, y_pred))
error_train = np.log(mean_absolute_error(y_train, y_pred_train))
print('col', columns_comb, 'error', error, 'error_train', error_train)

col ['tertiary_angle_0', 'atom_1_bond_lengths_mean', 'inv_dist1R', 'yukawa_H.y', 'd_4_3', 'mulliken_atom_0', 'dist_O_0_x', 'dist_to_type_0_mean', 'tertiary_distance_2', 'dist_C_0_y', 'inv_dist0R', 'atom_index_1_cycle_size_mean', 'd_3_2', 'd_4_2', 'dist_O_0_y', 'inv_distPE', 'eem_1', 'dist_median_bond_y', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_C_2_y', 'dist_H_1_x', 'dist_O_1_x', 'cos_center1', 'dist_C_2_x', 'dist_N_0_y', 'mulliken_atom_1', 'dist_C_3_x', 'dist_to_type_1_mean', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'tertiary_angle_1', 'yukawa_H.x', 'molecule_atom_index_0_dist_max_div', 'dist_C_1_y', 'dist_N_0_x', 'tertiary_angle_2', 'tertiary_angle_4', 'molecule_atom_index_1_dist_min_diff', 'dist_to_type_mean', 'tertiary_atom_0', 'd_5_1', 'dist_H_1_y', 'tertiary_distance_3', 'adN1', 'tertiary_angle_5', 'd_4_0', 'dist_C_3_y', 'tertiary_distance_5', 'atom_index_farthest_0', 'distC0', 'dist_C_4_y', 'adC1', 'tertiary_distance_4', 'cos_f0', 'distC1', 'd_3_0', 'tertiary_angle_6', '

In [ ]:
-0.4239748046183735 -1.4237774481571122 10
-0.43376108984538503 -1.4402760557422896 8
-0.40170720756899114 -1.4836995674002746 5
-0.368376392396792 -1.5932999128808094 3

In [88]:
columns = df_train2.columns.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'])

X_data = df_train2.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'], axis=1).values.astype('float32')
y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

group_kfold = GroupKFold(n_splits=8)
splits = group_kfold.split(X_data, y_data, df_train2.group)
for train_index, test_index in splits:
    break
X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]

model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
y_pred = model.predict(X_val)
y_pred_train = model.predict(X_train)
error = np.log(mean_absolute_error(y_val, y_pred))
error_train = np.log(mean_absolute_error(y_train, y_pred_train))
print(error, error_train)

perm = PermutationImportance(model, random_state=42).fit(X_val, y_val)
df_feature_importances_i2 = eli5.explain_weights_dfs(perm)['feature_importances']
df_feature_importances_i2['feature2'] = df_feature_importances_i2['feature'].apply(lambda x : columns[int(x.replace('x',''))])
df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['weight'], ascending=False)
df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)

-0.4324559369556779 -1.436310972165935


In [89]:
columns_comb =[]
best_error = 999
his = []
for col in df_feature_importances_i2.feature2.tolist():
    
    X_data = df_train2[[col]+columns_comb].values.astype('float32')
    y_data = df_train2['scalar_coupling_constant'].values.astype('float32')
    
    group_kfold = GroupKFold(n_splits=8)
    splits = group_kfold.split(X_data, y_data, df_train2.group)
    for train_index, test_index in splits:
        break
    X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]
    
    model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
    model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
    y_pred = model.predict(X_val)
    y_pred_train = model.predict(X_train)
    error = np.log(mean_absolute_error(y_val, y_pred))
    error_train = np.log(mean_absolute_error(y_train, y_pred_train))
    print('col', col, len(columns_comb), 'error', error, 'error_train', error_train)
    his.append({'col':[col]+columns_comb, 'error':error, 'error_train':error_train})
    
    if error < best_error:
        best_error = error
        columns_comb += [col]
    
    if len(columns_comb)>70:
        break
    

col tertiary_angle_0 0 error 1.6222798164456236 error_train 1.6293130461726788
col inv_dist1R 1 error 1.3769224028122156 error_train 1.3682046457081336
col d_4_3 2 error 1.0526559798074329 error_train 0.9622265150615955
col yukawa_H.y 3 error 0.8880966852477664 error_train 0.7545132450913669
col mulliken_atom_0 4 error 0.6699714302742804 error_train 0.4381590087846014
col dist_to_type_mean 5 error 0.5723210796938699 error_train 0.31897853564242257
col dist_O_0_x 6 error 0.35635604998565557 error_train 0.07690433869670563
col atom_1_n_bonds 7 error 0.3479806235557046 error_train 0.06708991647819564
col dist_to_type_1_mean 8 error 0.34027981596648077 error_train 0.057885699625021256
col atom_1_bond_lengths_mean 9 error 0.20992680506741004 error_train -0.061747579339364166
col dist_xyz 10 error 0.20773890811152795 error_train -0.05768874175371354
col dist_C_0_y 11 error 0.15185049041648496 error_train -0.14306861964440287
col dist_to_type_std 12 error 0.15375433387730386 error_train -0.14

col sd_molecule_atom_0_dist_xyz 61 error -0.4919546730012977 error_train -1.0509517773836077
col atom_5 61 error -0.4890540271745161 error_train -1.0451622615139409
col atom_index_0_sv_3 61 error -0.4963120573883877 error_train -1.0610702231404057
col inv_distP 62 error -0.49318824241211956 error_train -1.0614384839286077
col rc_C 62 error -0.49830299083243346 error_train -1.0773153469498888
col d_6_1 63 error -0.4894647430938032 error_train -1.0698224821629696
col d_8_1 63 error -0.4903476909384109 error_train -1.0776367466227816
col atom_index_0_sv_4 63 error -0.4959875676530493 error_train -1.089228245941935
col d_7_1 63 error -0.4859505832170369 error_train -1.071266218805379
col atom_index_0_sv_1 63 error -0.49285115059044254 error_train -1.086352052961335
col tertiary_atom_3 63 error -0.49040061219772113 error_train -1.0708975465615733
col cos_f1 63 error -0.5002637449457015 error_train -1.0901288969846596
col d_7_0 64 error -0.4972107634550171 error_train -1.0948819597623192
col

col molecule_atom_index_1_dist_max_diff 67 error -0.5038497510449675 error_train -1.1052230933637575
col tertiary_angle_12 67 error -0.5006790960752652 error_train -1.1225317411661033
col atom_index_1_dist_q.99 67 error -0.5019650905250351 error_train -1.1215623925888072
col atom_index_1_fiedler_eig 67 error -0.5004717035466232 error_train -1.1126256231821572
col adH2 67 error -0.5020626440704913 error_train -1.0996522236829769
col mulliken_min 67 error -0.4985403598207949 error_train -1.1120304697935761
col tertiary_atom_8 67 error -0.5035643875969926 error_train -1.1096691166947008
col atom_index_0_fiedler_eig 67 error -0.5001937769057281 error_train -1.1186863239693425
col tertiary_distance_13 67 error -0.4978514000970518 error_train -1.100724171957628
col yukawa_O.x 67 error -0.503149493115294 error_train -1.1022745443558095
col molecule_name.1 67 error -0.5024691485382106 error_train -1.1141206786580724
col vander_O.x 67 error -0.497348869017027 error_train -1.1035197890009252
col

col molecule_dist_mean_no_bond_x 67 error -0.494976657819703 error_train -1.1096757803639141
col molecule_atom_index_0_dist_std_div 67 error -0.4974255694811223 error_train -1.109277800691294
col Cv 67 error -0.49663169020192016 error_train -1.1091696454036963
col dist_median_no_bond_x 67 error -0.49729949753123864 error_train -1.108047356354795
col molecule_atom_index_0_y_1_std 67 error -0.5038901417676547 error_train -1.1139378272859617
col structure_y_1 67 error -0.5021603832586528 error_train -1.1239925544803577
col molecule_atom_1_dist_std_diff 67 error -0.5022273666293195 error_train -1.1155712253897183
col molecule_atom_index_1_dist_mean_div 67 error -0.5059505565888474 error_train -1.114228520597563
col range_dist_x 67 error -0.502830970412377 error_train -1.1286014472071595
col tertiary_angle_17 67 error -0.4998741252679 error_train -1.1068285621255125
col dist_F_0_y 67 error -0.5087888646847517 error_train -1.1085684179857822
col structure_x_1 68 error -0.500830656479518 erro

col dist_xy 68 error -0.5004906837105771 error_train -1.1108043992844692
col tertiary_atom_12 68 error -0.5005180560132533 error_train -1.1080521685796514
col atom_index_0_p_lap_core 68 error -0.49967140502891894 error_train -1.1132839033101694
col tertiary_atom_13 68 error -0.5053244666211033 error_train -1.1142339956235725
col dist_to_type_mean_yz 68 error -0.502770958637726 error_train -1.1138566856407415
col dist_mean_bond_y 68 error -0.5067659085663165 error_train -1.104472462214877
col tertiary_angle_19 68 error -0.5009783479764099 error_train -1.1060569058859087
col c_z 68 error -0.5031241947803698 error_train -1.1146866010878864
col dist_x 68 error -0.5014339886041407 error_train -1.111321903085337
col z_dist 68 error -0.5020520676432993 error_train -1.1158614504744202
col atom_index_0_t_dist_factors_0 68 error -0.5017064576379667 error_train -1.1104905908219254
col tertiary_angle_20 68 error -0.5043191194268607 error_train -1.1006907901293295
col atom_index_1_p_dist_factors 68

In [90]:
columns_comb

['tertiary_angle_0',
 'inv_dist1R',
 'd_4_3',
 'yukawa_H.y',
 'mulliken_atom_0',
 'dist_to_type_mean',
 'dist_O_0_x',
 'atom_1_n_bonds',
 'dist_to_type_1_mean',
 'atom_1_bond_lengths_mean',
 'dist_xyz',
 'dist_C_0_y',
 'd_3_2',
 'atom_index_1_ hybridization',
 'atom_index_1_cycle_size_mean',
 'dist_O_0_y',
 'eem_1',
 'inv_distPE',
 'd_4_2',
 'inv_distPR',
 'dist_no_bond_min_y',
 'dist_H_2_x',
 'dist_H_1_x',
 'tertiary_distance_2',
 'dist_C_3_x',
 'dist_O_1_x',
 'atom_1_bond_lengths_std',
 'dist_C_2_y',
 'dist_C_2_x',
 'mulliken_atom_1',
 'cos_center1',
 'dist_O_1_y',
 'tertiary_angle_3',
 'dist_H_2_y',
 'dist_N_0_y',
 'dist_C_1_y',
 'inv_dist1E',
 'distance_y',
 'tertiary_angle_2',
 'dist_N_0_x',
 'd_2_1',
 'molecule_atom_index_0_dist_max_div',
 'adC1',
 'adN1',
 'd_4_0',
 'dist_C_3_y',
 'atom_3',
 'distC0',
 'tertiary_distance_4',
 'tertiary_angle_5',
 'd_5_1',
 'molecule_atom_index_1_dist_min_diff',
 'dist_C_4_y',
 'dist_H_0_y',
 'cos_f0',
 'd_5_0',
 'tertiary_distance_3',
 'd_5_2',


In [91]:
df_train2 = pd.read_pickle(f'{file_folder}/df_train2.gzde', compression='gzip')
df_train2_plus = pd.read_pickle(f'{file_folder}/df_train2_plus.gzde', compression='gzip')
df_train2_plus = df_train2_plus.rename(columns={'id':'index'})

df_train2 = df_train2[df_train2['type']==1]
df_train2 = pd.merge(df_train2, df_train2_plus, how='left', on='index')
df_train2 = df_train2.fillna(0)

In [94]:
columns = df_train2.columns.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'])

X_data = df_train2.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'], axis=1).values.astype('float32')
y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

group_kfold = GroupKFold(n_splits=8)
splits = group_kfold.split(X_data, y_data, df_train2.group)
for train_index, test_index in splits:
    break
X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]

model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
y_pred = model.predict(X_val)
y_pred_train = model.predict(X_train)
error = np.log(mean_absolute_error(y_val, y_pred))
error_train = np.log(mean_absolute_error(y_train, y_pred_train))
print(error, error_train)

perm = PermutationImportance(model, random_state=42).fit(X_val, y_val)
df_feature_importances_i2 = eli5.explain_weights_dfs(perm)['feature_importances']
df_feature_importances_i2['feature2'] = df_feature_importances_i2['feature'].apply(lambda x : columns[int(x.replace('x',''))])
df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['weight'], ascending=False)
df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)

-1.045524140353588 -3.578226504262563


In [95]:
columns_comb =[]
best_error = 999
his = []
for col in df_feature_importances_i2.feature2.tolist():
    
    X_data = df_train2[[col]+columns_comb].values.astype('float32')
    y_data = df_train2['scalar_coupling_constant'].values.astype('float32')
    
    group_kfold = GroupKFold(n_splits=8)
    splits = group_kfold.split(X_data, y_data, df_train2.group)
    for train_index, test_index in splits:
        break
    X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]
    
    model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
    model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
    y_pred = model.predict(X_val)
    y_pred_train = model.predict(X_train)
    error = np.log(mean_absolute_error(y_val, y_pred))
    error_train = np.log(mean_absolute_error(y_train, y_pred_train))
    print('col', col, len(columns_comb), 'error', error, 'error_train', error_train)
    his.append({'col':[col]+columns_comb, 'error':error, 'error_train':error_train})
    
    if error < best_error:
        best_error = error
        columns_comb += [col]
        j = 0
    else:
        j += 1
    
    if (len(columns_comb)>=70) or (j>=100):
        break
    

col dist_H_0_y 0 error 1.081751973702698 error_train 1.0550294977466406
col d_3_2 1 error 0.6005353595852118 error_train 0.49221504462929516
col dist_C_0_y 2 error 0.07807593070742562 error_train -0.19800769806785656
col atom_index_1_ aromatic 3 error 0.01719642781707902 error_train -0.3501746232959124
col atom_1_bond_lengths_mean 4 error -0.23493842088244496 error_train -0.7217087512759597
col bond_atom 5 error -0.2625240684316153 error_train -0.7541296419922382
col inv_dist1R 6 error -0.3680639812043885 error_train -0.8385121530493932
col d_3_1 7 error -0.3973806357142903 error_train -0.8972256047903716
col mulliken_atom_0 8 error -0.5487510827793718 error_train -1.1778260376962102
col dist_H_0_x 9 error -0.706022820177178 error_train -1.4175115229195456
col dist_O_0_y 10 error -0.7513907884756217 error_train -1.5692261567853103
col dist_C_1_x 11 error -0.8769879159143399 error_train -1.7417710394301336
col tertiary_angle_0 12 error -0.8897162953581453 error_train -1.8159052093935795

col tertiary_distance_1 36 error -1.1806053035026383 error_train -2.3788887706000663
col cos_f1 36 error -1.1900787052113657 error_train -2.382614369258613
col molecule_atom_index_0_dist_std_diff 36 error -1.1889738455769332 error_train -2.436202936914872
col atom_index_0_sv_min 36 error -1.1590918988350472 error_train -2.427956629774231
col dist_H_3_y 36 error -1.1794105581188876 error_train -2.4590630406211953
col tertiary_distance_7 36 error -1.182736220136069 error_train -2.4484288803623606
col molecule_type_dist_min 36 error -1.1767523586553024 error_train -2.384211674872186
col mu 36 error -1.1780426689678318 error_train -2.4898321257310876
col tertiary_atom_0 36 error -1.1897119990437421 error_train -2.4961754856557206
col atom_index_0_sv_0 36 error -1.1721192738123412 error_train -2.41966950639852
col d_5_3 36 error -1.1923051378673628 error_train -2.3750046889884633
col d_6_2 36 error -1.1738135843908206 error_train -2.4379475439104783
col range_dist_bond_y 36 error -1.1690212

In [96]:
columns_comb

['dist_H_0_y',
 'd_3_2',
 'dist_C_0_y',
 'atom_index_1_ aromatic',
 'atom_1_bond_lengths_mean',
 'bond_atom',
 'inv_dist1R',
 'd_3_1',
 'mulliken_atom_0',
 'dist_H_0_x',
 'dist_O_0_y',
 'dist_C_1_x',
 'tertiary_angle_0',
 'dist_C_1_y',
 'vander_C.y',
 'dist_H_1_y',
 'mulliken_atom_1',
 'inv_dist0R',
 'd_1_0',
 'tertiary_distance_0',
 'tertiary_angle_2',
 'atom_index_1_explicit_valence',
 'dist_N_0_y',
 'inv_distPR',
 'dist_C_2_x',
 'vander_H.x',
 'd_4_2',
 'atom_index_0_eigv_max',
 'tertiary_distance_2',
 'dist_H_1_x',
 'dist_N_1_x',
 'dist_C_3_x',
 'cos_f0',
 'atom_index_1_sv_2',
 'max_molecule_atom_0_dist_xyz',
 'd_2_1']

In [ ]:
for i in [4,5,6,7]:
    df_train2 = pd.read_pickle(f'{file_folder}/df_train2.gzde', compression='gzip')
    df_train2_plus = pd.read_pickle(f'{file_folder}/df_train2_plus.gzde', compression='gzip')
    df_train2_plus = df_train2_plus.rename(columns={'id':'index'})

    df_train2 = df_train2[df_train2['type']==i]
    df_train2 = pd.merge(df_train2, df_train2_plus, how='left', on='index')
    df_train2 = df_train2.fillna(0)
    
    columns = df_train2.columns.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'])

    X_data = df_train2.drop(['index', 'group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso'], axis=1).values.astype('float32')
    y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

    group_kfold = GroupKFold(n_splits=8)
    splits = group_kfold.split(X_data, y_data, df_train2.group)
    for train_index, test_index in splits:
        break
    X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]

    model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
    model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
    y_pred = model.predict(X_val)
    y_pred_train = model.predict(X_train)
    error = np.log(mean_absolute_error(y_val, y_pred))
    error_train = np.log(mean_absolute_error(y_train, y_pred_train))
    print('init', i, error, error_train)

    perm = PermutationImportance(model, random_state=42).fit(X_val, y_val)
    df_feature_importances_i2 = eli5.explain_weights_dfs(perm)['feature_importances']
    df_feature_importances_i2['feature2'] = df_feature_importances_i2['feature'].apply(lambda x : columns[int(x.replace('x',''))])
    df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['weight'], ascending=False)
    df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)
    
    columns_comb =[]
    best_error = 999
    his = []
    for col in df_feature_importances_i2.feature2.tolist():

        X_data = df_train2[[col]+columns_comb].values.astype('float32')
        y_data = df_train2['scalar_coupling_constant'].values.astype('float32')

        group_kfold = GroupKFold(n_splits=8)
        splits = group_kfold.split(X_data, y_data, df_train2.group)
        for train_index, test_index in splits:
            break
        X_train, X_val, y_train, y_val = X_data[train_index], X_data[test_index], y_data[train_index], y_data[test_index]

        model = lgb.LGBMRegressor(**LGB_PARAMS, n_estimators=1500, n_jobs = -1)
        model.fit(X_train, y_train,  eval_set=[(X_train, y_train), (X_val, y_val)], eval_metric='mae', verbose=False, early_stopping_rounds=200)
        y_pred = model.predict(X_val)
        y_pred_train = model.predict(X_train)
        error = np.log(mean_absolute_error(y_val, y_pred))
        error_train = np.log(mean_absolute_error(y_train, y_pred_train))
        print('col', col, len(columns_comb), 'error', error, 'error_train', error_train)
        his.append({'col':[col]+columns_comb, 'error':error, 'error_train':error_train})

        if error < best_error:
            best_error = error
            columns_comb += [col]
            j = 0
        else:
            j += 1

        if (len(columns_comb)>=70) or (j>=100):
            print(columns_comb)
            break
    

init 4 -1.7209448613538423 -4.634338798915695
col d_3_1 0 error 0.5303010881908956 error_train 0.5265754225870725
col dist_H_1_x 1 error 0.09337929063177976 error_train 0.01434376211829868
col d_5_0 2 error -0.43390038861652086 error_train -0.7365402575581361
col d_4_0 3 error -0.8242287878098185 error_train -1.2240698395730305
col yukawa_H.x 4 error -0.8573470134371305 error_train -1.364166013819757
col inv_dist0 5 error -0.9685407472429141 error_train -1.615406298677814
col d_6_0 6 error -1.1764654098146645 error_train -1.8786303916849456
col d_4_1 7 error -1.3666404698858718 error_train -2.0905596893506067
col cos_c0 8 error -1.4672456431192764 error_train -2.263491546585578
col atom_3 9 error -1.4967113653752862 error_train -2.282500733554745
col dist_C_0_y 10 error -1.571925780803937 error_train -2.4222939691788716
col molecule_atom_index_0_dist_std_div 11 error -1.5839250503696614 error_train -2.451341627294883
col cos_c0_c1 12 error -1.6455990169470411 error_train -2.51819848725

col inv_dist0E 43 error -1.9098158260643003 error_train -3.0251654989901415
col inv_dist1R 43 error -1.8934566905103327 error_train -3.037385313978916
col distance_center1 43 error -1.8910127747114531 error_train -3.0603405256622307
col atom_1_n_bonds 43 error -1.9030307869023526 error_train -3.0246909254604613
col d_7_2 43 error -1.8997894003797833 error_train -3.0607781361056143
col yukawa_H.y 43 error -1.8878676418976124 error_train -3.035346205837835
col inv_dist1 43 error -1.898412841590142 error_train -3.0378149438157545
col dist_H_2_x 43 error -1.9021211326259435 error_train -3.0493123936669626
col atom_index_1_n_cycle 43 error -1.9119829540645437 error_train -3.0229353984521654
col tertiary_angle_4 43 error -1.895204640856336 error_train -3.045360764473385
col dist_to_type_mean 43 error -1.9137102029528517 error_train -3.0196042770972347
col dist_to_type_1_mean 43 error -1.9010049680553858 error_train -3.020804927891549
col d_8_2 43 error -1.9035987806196994 error_train -3.0525

col freqs_max 45 error -1.8923401411872978 error_train -3.086297416943557
col dist_1_neg_2 45 error -1.906752300647843 error_train -3.061041732650863
col link1 45 error -1.907779399374713 error_train -3.068989139482911
col dist_median_x 45 error -1.8883909342294614 error_train -3.069281438914519
col atom_index_1_eigv_gap 45 error -1.8998353138394355 error_train -3.070000110080792
col molecule_dist_mean 45 error -1.892068355390327 error_train -3.053611707966008
col inv_dist1E 45 error -1.892929140482324 error_train -3.0574379114996635
col d_9_1 45 error -1.9022377854322783 error_train -3.0732381102995174
col rc_B 45 error -1.9024863094449265 error_train -3.0781971709309035
col dist_neg_2 45 error -1.9170000170188397 error_train -3.0566875281289825
col dist_1_neg_3 45 error -1.9046938219634724 error_train -3.071024091213973
col distance_center0 45 error -1.8840323980388003 error_train -3.069334584395132
col molecule_dist_std_y 45 error -1.8785594055722077 error_train -3.0774323083739183


col dist_N_2_x 46 error -1.8963472711954357 error_train -3.0339547159468383
col atom_index_0_eigv_min 46 error -1.8960175842327454 error_train -3.1014099187677426
col dist_O_1_y 46 error -1.9157981703962381 error_train -3.0627556520891615
col dz 46 error -1.9003189579223234 error_train -3.0911221399574704
col dimension_z 46 error -1.8880546069682698 error_train -3.093404902938179
col range_dist_no_bond_y 46 error -1.8895398444909512 error_train -3.091948272785836
col atom_index_1_eigv_min 46 error -1.897183571012263 error_train -3.0963308066354327
col range_dist_no_bond_x 46 error -1.9014149126520858 error_train -3.090594339822902
col molecule_atom_index_1_dist_mean_diff 46 error -1.9082805260126183 error_train -3.072650564487505
col dist_N_1_y 46 error -1.9130493683000693 error_train -3.0560129840141133
col dist_std_no_bond_x 46 error -1.8956185793274918 error_train -3.0910834209926192
col range_dist_x 46 error -1.9013434527251438 error_train -3.092810967842444
col molecule_atom_index

In [ ]:
init 0 -0.4324559369556779 -1.436310972165935
['tertiary_angle_0', 'inv_dist1R', 'd_4_3', 'yukawa_H.y', 'mulliken_atom_0', 'dist_to_type_mean', 'dist_O_0_x', 'atom_1_n_bonds', 'dist_to_type_1_mean', 'atom_1_bond_lengths_mean', 'dist_xyz', 'dist_C_0_y', 'd_3_2', 'atom_index_1_ hybridization', 'atom_index_1_cycle_size_mean', 'dist_O_0_y', 'eem_1', 'inv_distPE', 'd_4_2', 'inv_distPR', 'dist_no_bond_min_y', 'dist_H_2_x', 'dist_H_1_x', 'tertiary_distance_2', 'dist_C_3_x', 'dist_O_1_x', 'atom_1_bond_lengths_std', 'dist_C_2_y', 'dist_C_2_x', 'mulliken_atom_1', 'cos_center1', 'dist_O_1_y', 'tertiary_angle_3', 'dist_H_2_y', 'dist_N_0_y', 'dist_C_1_y', 'inv_dist1E', 'distance_y', 'tertiary_angle_2', 'dist_N_0_x', 'd_2_1', 'molecule_atom_index_0_dist_max_div', 'adC1', 'adN1', 'd_4_0', 'dist_C_3_y', 'atom_3', 'distC0', 'tertiary_distance_4', 'tertiary_angle_5', 'd_5_1', 'molecule_atom_index_1_dist_min_diff', 'dist_C_4_y', 'dist_H_0_y', 'cos_f0', 'd_5_0', 'tertiary_distance_3', 'd_5_2', 'tertiary_atom_1', 'dist_C_4_x', 'cos_c0_f0', 'atom_index_0_sv_3', 'rc_C', 'cos_f1', 'tertiary_angle_8', 'dist_O_2_y', 'max_molecule_atom_1_dist_xyz', 'dist_F_0_y', 'atom_index_1_ aromatic', 'tertiary_angle_26', 'type_0']
init 1 -1.045524140353588 -3.578226504262563
['dist_H_0_y', 'd_3_2', 'dist_C_0_y', 'atom_index_1_ aromatic', 'atom_1_bond_lengths_mean', 'bond_atom', 'inv_dist1R', 'd_3_1', 'mulliken_atom_0', 'dist_H_0_x', 'dist_O_0_y', 'dist_C_1_x', 'tertiary_angle_0', 'dist_C_1_y', 'vander_C.y', 'dist_H_1_y', 'mulliken_atom_1', 'inv_dist0R', 'd_1_0', 'tertiary_distance_0', 'tertiary_angle_2', 'atom_index_1_explicit_valence', 'dist_N_0_y', 'inv_distPR', 'dist_C_2_x', 'vander_H.x', 'd_4_2', 'atom_index_0_eigv_max', 'tertiary_distance_2', 'dist_H_1_x', 'dist_N_1_x', 'dist_C_3_x', 'cos_f0', 'atom_index_1_sv_2', 'max_molecule_atom_0_dist_xyz', 'd_2_1']
init 2 -1.316571636180272 -2.044293659651798
['tertiary_atom_0', 'inv_dist0', 'dist_no_bond_min_x', 'atom_index_1_ hybridization', 'tertiary_angle_0', 'tertiary_angle_1', 'dist_O_0_x', 'cos_c0', 'd_5_2', 'tertiary_atom_1', 'cos_f0', 'dist_H_0_x', 'd_3_1', 'atom_index_1_degree', 'dist_C_0_y', 'adC2', 'dist_C_3_x', 'vander_O.y', 'mulliken_atom_1', 'atom_7', 'tertiary_angle_2', 'd_2_1', 'atom_3', 'd_5_1', 'd_6_2', 'd_4_1', 'tertiary_atom_2', 'molecule_atom_index_1_dist_min_diff', 'd_4_2', 'dist_C_2_x', 'cos_c0_f0', 'd_6_0', 'dist_O_0_y', 'd_4_3', 'd_3_0', 'd_7_0', 'd_3_2', 'inv_dist0R', 'atom_8', 'dist_C_1_x', 'd_6_1', 'd_2_0', 'd_8_1', 'mulliken_atom_0', 'dist_N_0_x', 'atom_4', 'tertiary_distance_2', 'd_7_2', 'dist_C_0_x', 'atom_1_bond_lengths_mean', 'dist_C_1_y', 'bond_atom', 'd_7_1', 'd_4_0', 'distC0', 'atom_index_1_cycle_size_mean', 'cos_c0_c1', 'tertiary_angle_3', 'dist_O_1_x', 'atom_index_1_n_cycle', 'max_molecule_atom_0_dist_xyz', 'molecule_atom_index_0_dist_max_div', 'atom_5', 'gap', 'cos_c1', 'dist_N_0_y', 'd_6_3', 'dist_C_3_y', 'inv_distP', 'dist_C_4_y']
init 3 -1.9251364757149136 -3.3697638772965974
['cos_c0', 'd_4_3', 'cos_c0_c1', 'molecule_atom_index_0_dist_min_diff', 'tertiary_atom_1', 'd_3_2', 'd_1_0', 'dist_H_0_y', 'mulliken_atom_0', 'mulliken_atom_1', 'dist_N_0_x', 'link0', 'tertiary_atom_2', 'dist_C_1_y', 'dist_C_1_x', 'cos_f0', 'dist_C_0_y', 'cos_f1', 'd_3_1', 'tertiary_distance_1', 'dist_O_0_y', 'cos_f0_f1', 'adC1', 'd_5_3', 'inv_distP', 'edge_4', 'd_6_2', 'dist_N_0_y', 'tertiary_distance_2', 'dist_O_0_x', 'cos_c1_f1', 'd_3_0', 'd_5_2', 'dist_C_0_x', 'adN1', 'cos_c0_f0', 'd_4_1', 'max_distance_y', 'dist_C_2_y', 'atom_5', 'adC3', 'dist_to_type_1_mean', 'vander_H.x', 'dist_C_3_y', 'dist_H_3_x', 'molecule_atom_index_0_dist_max_div', 'atom_7', 'dist_C_3_x', 'd_5_1', 'dist_H_3_y', 'atom_index_0_eigv_max', 'atom_6', 'dist_H_2_x', 'atom_index_1_sv_0', 'molecule_atom_index_1_dist_std_div', 'link1']
init 4 -1.7209448613563647 -4.63433879893567
['d_3_1', 'dist_H_1_x', 'd_5_0', 'd_4_0', 'yukawa_H.x', 'inv_dist0', 'd_6_0', 'd_4_1', 'cos_c0', 'atom_3', 'dist_C_0_y', 'molecule_atom_index_0_dist_std_div', 'cos_c0_c1', 'd_4_2', 'min_molecule_atom_0_dist_xyz', 'sd_molecule_atom_0_dist_xyz', 'd_2_1', 'adC2', 'd_3_0', 'dist_C_1_y', 'd_4_3', 'dist_H_0_x', 'vander_C.x', 'd_5_3', 'dist_H_1_y', 'tertiary_distance_3', 'd_2_0', 'dist_O_0_x', 'd_5_1', 'dist_O_0_y', 'adC3', 'inv_dist0R', 'dist_C_3_y', 'atom_index_1_ hybridization', 'cos_f0', 'dist_C_2_x', 'd_5_2', 'd_6_1', 'dist_C_0_x', 'atom_1_bond_lengths_min', 'mulliken_atom_1', 'distance_farthest_0', 'tertiary_distance_1', 'min_molecule_atom_1_dist_xyz', 'yukawa_O.y', 'atom_0_bond_lengths_max']
init 5 -1.236895026643203 -1.8522039705242035
